In [ ]:
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

import matplotlib.pyplot as plt
import numpy as np
import _pickle as pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import backend as K, optimizers
from tensorflow.keras.layers import Activation, Dropout, Layer, Conv1D, Dense, BatchNormalization
from tqdm.notebook import trange
import time
import pandas as pd
from datetime import datetime, timedelta
from sklearn import preprocessing

Mounted at /content/drive/


# Helper functions

In [ ]:
def compute_coverage_len(y_test, y_lower, y_upper,verbose=False):
    """ 
    Compute average coverage and length of prediction intervals
    """
    in_the_range = np.sum((y_test >= y_lower) & (y_test <= y_upper))
    coverage = in_the_range / (y_test.shape[0]*y_test.shape[1])
    avg_length = np.mean(abs(y_upper - y_lower))
    avg_length = avg_length/(y_test.max()-y_test.min())
    if verbose==True:
      print("PI coverage:",coverage,",PI avg. length",avg_length)
    return coverage, avg_length


def asym_nonconformity(label,low,high):
    """
    Compute the asymetric conformity score
    
    credit - https://github.com/yromano/cqr
    """
    y_lower = low
    y_upper = high
    error_high = label - y_upper 
    error_low = y_lower - label
    return error_low, error_high


def transform_to_windows(data_org, station):
    """
    Create dataframe with hours as columns 

    credit - https://github.com/nicholasjhana/short-term-energy-demand-forecasting
    """
    #from the original datetime index create new columns with each of the year, month, day, and hour.
    data = data_org.copy()
    data.loc[:,'year'] = data.index.year
    data.loc[:,'month'] = data.index.month
    data.loc[:,'day'] = data.index.day
    data.loc[:,'hours'] = data.index.hour
    #construct datetimes from the split year, month, day columns
    data.loc[:,'date'] = pd.to_datetime(data.loc[:,['year', 'month', 'day']], format='%Y-%m-%d', errors='ignore')
    #set the index to dates only
    data = data.set_index(pd.DatetimeIndex(data['date']))
    #drop non target columns 
    data = data.loc[:,[station, 'hours']]
    #pivot the table into the format Date h0, h1, ...h23
    data = data.pivot(columns='hours', values=station)
    data = data.dropna()
    return data


def split_sequences(sequences, n_steps):
    """
    Split data into observations and labels 

    credit - https://github.com/nicholasjhana/short-term-energy-demand-forecasting
    """
    max_step=n_steps
    n_steps+=1
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + max_step
        #create a list with the indexes we want to include in each sample
        slices = [x for x in range(end_ix-1,end_ix-n_steps, -1)] + [y for y in range(end_ix-n_steps, i, -7)]
        #reverse the slice indexes
        slices = list(reversed(slices))
        # check if we are beyond the dataset
        if end_ix > len(sequences)-1:
            break
        # gather input and output parts of the pattern
        seq_x = sequences[slices, :]
        seq_y = sequences[end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    X = np.array(X)
    X = np.reshape(X,(X.shape[0],X.shape[1]*X.shape[2],1))
    y = np.array(y)
    return X, y


def normalize_data(data):
    if len(data.shape) == 3:
      data_r = data.reshape(-1,data.shape[1]*data.shape[-1])
      scaler = preprocessing.MinMaxScaler().fit(data_r)
      data_norm = scaler.transform(data_r).reshape(-1,data.shape[1],data.shape[-1])
    else:
      scaler = preprocessing.MinMaxScaler().fit(data)
      data_norm = scaler.transform(data)
    return data_norm, scaler


def create_datasets(data):
    """
    Create input-output pairs from a dataframe with observations
    """
    df_data=[]
    cols = data.columns
    for i,s in enumerate(cols): 
        if i == 0:
            X,Y = split_sequences(transform_to_windows(data,s).values,7)
        else:
            x,y = split_sequences(transform_to_windows(data,s).values,7)
            X = np.append(X,x,-1)
    return X,Y



def create_ensemble_datasets(df_train,B):
  train=[]
  train_ensemble=[]
  for j,s in enumerate(df_train.columns):
    df_t = transform_to_windows(df_train,s)
    X_full,Y_full = split_sequences(df_t.values,7)
    X_norm, scaler_x = normalize_data(X_full)
    Y_norm, scaler_y = normalize_data(Y_full)
    # train.append([X_norm,Y_norm,scaler_x,scaler_y])
    train_s = []
    for i in range(B):
      sb_size = int(np.floor(len(df_t)/B))
      X,Y = split_sequences(df_t[i*sb_size:i*sb_size+sb_size].values,7)
      X_r = X.reshape(X.shape[0],X.shape[1]*X.shape[2])
      X = scaler_x.transform(X_r)
      X = X.reshape((X.shape[0],X_full.shape[1],X_full.shape[2]))
      Y = scaler_y.transform(Y)
      train_s.append([X,Y])
    train_ensemble.append(train_s)
    # combine all features 
  for j in range(B):
      for i in range(len(df_train.columns)):
          if i == 0:
              etrain_x, etrain_y= train_ensemble[i][j] 
          else:
              etrain_xx, etrain_yy = train_ensemble[i][j]
              etrain_x = np.append(etrain_x,etrain_xx,axis=-1)
      train.append([etrain_x,etrain_y])
  return train



class LSTM_stateful(keras.Model):
    def __init__(self, parameters):
      """ Initialization
      Parameters
      ---------
      parameters : list containing tcn network parameters
      """
      super(LSTM_stateful, self).__init__()
      self.model_type = 'lstm'
      self.n_lags = parameters['n_lags']
      self.n_feat = parameters['n_feat']
      self.cells = parameters['cells']
      self.n_layers = parameters['n_layers']
      self.quantiles = parameters['quantiles']
      self.l2_l = parameters['l2']
      self.batch_s = parameters['batch_s']
      self.target_coverage = (self.quantiles[-1]-self.quantiles[0])
      self.num_quantiles = len(self.quantiles)
      self.lstm = keras.models.Sequential(name='lstm_stateful')
      
      with K.name_scope(self.name): 
        if self.n_layers > 1:   
              for i in range(self.n_layers-1):
               self.lstm.add(keras.layers.LSTM(self.cells,stateful=True, return_sequences=True, kernel_regularizer=keras.regularizers.l2(self.l2_l)))
        
        self.lstm.add(keras.layers.LSTM(self.cells, stateful=True, kernel_regularizer=keras.regularizers.l2(self.l2_l)))
        self.lstm.add(keras.layers.Dense(24, kernel_regularizer=keras.regularizers.l2(self.l2_l)))
        self.out_quantiles = keras.layers.Dense(len(self.quantiles), kernel_regularizer=keras.regularizers.l2(self.l2_l))

    def call(self,x,training=None): 
      output = self.lstm(x) 
      output = tf.reshape(output,(output.shape[0],output.shape[1],1))
      output_quantiles = self.out_quantiles(output)
      return output_quantiles


def predict_stateful(obs,lab,model,batch_size,pad=False):
  # stateful
  max_batch_count_val = int(obs.shape[0] / batch_size)
  X_test = obs[0:max_batch_count_val*batch_size]
  y_test = lab[0:max_batch_count_val*batch_size]
  missing = batch_size - (obs.shape[0] - max_batch_count_val*batch_size)
  # pad with zeros 
  if pad == True:
    zero_pad_x = np.zeros((missing,obs.shape[1],obs.shape[2]))
    zero_pad_y = np.zeros((missing,lab.shape[1]))
    X_test = obs
    y_test = lab
    X_test = np.append(np.array(X_test),zero_pad_x,axis=0)
    y_test = np.append(np.array(y_test),zero_pad_y,axis=0)
  val_data = tf.data.Dataset.from_tensor_slices((X_test, y_test))
  v_pred = []
  val_iter = val_data.batch(batch_size)
  for batch, (v_data,v_label) in enumerate(val_iter):
    valid_pred = model(v_data,training=False)
    v_pred.append(valid_pred)
  for p in range(len(v_pred)):
    if p == 0:
      va_pred = np.array(v_pred[p])
    else:
      va_pred = np.append(va_pred,v_pred[p],axis=0)
  if pad == True:
    va_pred = np.delete(va_pred,slice(len(va_pred)-missing,len(va_pred),1),axis=0)
  return va_pred


def train_model(model, train_x, train_y, test_x,test_y, trainer_params_list,early_stop = False):
    """ fit the model to the data
    Parameters:
    -----------
    model : network
    train_x : training obs
    train_y : training labels
    test_x : test obs
    test_y : test labels
    trainer_params_list : list of training parameters

    Returns:
    --------
    model : trained network
    """
    batch_size = trainer_params_list['batch_size']
    epochs = trainer_params_list['epoch_num']
    loss_func = trainer_params_list['loss_function']
    optimizer = trainer_params_list['optimizer']
    lr = trainer_params_list['lr']

    # early stopping par
    patience = trainer_params_list['patience']
    delta = trainer_params_list['delta']

    best_epoch = 0
    best_avg_length = 1e10
    best_coverage = 0

    train_loss, test_loss,test_cov,test_len = [],[],[],[]
    max_batch_count = int(train_x.shape[0] / batch_size)
    max_batch_count_val = int(test_x.shape[0] / batch_size)
    missing = batch_size - (train_x.shape[0] - max_batch_count*batch_size)

    X_train = train_x[0:max_batch_count*batch_size]
    y_train = train_y[0:max_batch_count*batch_size]
    X_test = test_x[0:max_batch_count_val*batch_size]
    y_test = test_y[0:max_batch_count_val*batch_size]

    train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    val_data = tf.data.Dataset.from_tensor_slices((X_test, y_test))

    ### The training process
    for epoch in trange(epochs):
        start=time.time()
        epoch_loss_avg = tf.keras.metrics.Mean()
        epoch_loss_avg_test = tf.keras.metrics.Mean()

        train_iter = train_data.batch(batch_size)
        val_iter = val_data.batch(batch_size)
        for batch, (conv_data,label) in enumerate(train_iter):
            with tf.GradientTape() as tape:
                output= model(conv_data, training=True) 
                loss = loss_func(label,output)
            grad = tape.gradient(loss,model.trainable_variables)
            optimizer.apply_gradients(zip(grad, model.trainable_variables))
            epoch_loss_avg.update_state(loss)
        train_loss.append(epoch_loss_avg.result())

        # evaluation
        v_pred = []
        for batch, (v_data,v_label) in enumerate(val_iter):
          valid_pred = model(v_data,training=False)
          v_pred.append(valid_pred)
          val_loss = loss_func(v_label,valid_pred)
          epoch_loss_avg_test.update_state(val_loss)
        test_loss.append(epoch_loss_avg_test.result())
        for p in range(len(v_pred)):
          if p == 0:
            va_pred = np.array(v_pred[p])
          else:
            va_pred = np.append(va_pred,v_pred[p],axis=0)
        coverage, avg_length = compute_coverage_len(y_test,va_pred[:,:,0],va_pred[:,:,-1])
        test_cov.append(coverage)
        test_len.append(avg_length)
        if (coverage >= model.target_coverage) and (avg_length < best_avg_length):
          best_avg_length = avg_length
          best_coverage = coverage
          best_epoch = epoch

        # EarlyStopping
        if early_stop == True:
          if len(test_loss) > patience:
              last_patience_epochs = [x + delta for x in test_loss[::-1][1:patience + 1]]
              current_metric = test_loss[::-1][0]
              if current_metric >= min(last_patience_epochs):
                break

    # plotting learning curves
    plt.figure()
    plt.plot(train_loss,label = 'train loss')
    plt.plot(test_loss,label = 'test loss')
    plt.legend()
    plt.show()
    if len(model.quantiles) >= 2:
      plt.figure()
      plt.plot(test_cov,label = 'test cov')
      plt.plot(test_len,label = 'test len')
      plt.hlines(model.target_coverage,0,len(test_cov))
      plt.ylim((0,1.5))
      plt.legend()
      plt.show()
    return model

class PinballLoss(keras.Model):
  """ Pinball loss for multiple quantiles
  """
  def __init__(self,quantiles):
    super(PinballLoss,self).__init__()
    self.quantiles = quantiles

  def call(self,labels,pred):
    """ Compute pinball loss between labels and pred 
    Parameters:
    ----------
    labels : true labels
    pred : predictions of true labels 

    Returns:
    -------
    total_loss : pinball loss value for all quantiles combined 
    """
    assert len(labels) == len(pred)
    loss = []
    for i,q in enumerate(self.quantiles):
      error = tf.subtract(labels,pred[:,:,i])
      loss_q = tf.reduce_mean(tf.maximum(q*error,(q-1)*error))
      loss.append(loss_q)
    L = tf.convert_to_tensor(loss)
    total_loss = tf.reduce_mean(L)
    return total_loss

# Dataset

In [ ]:
url = 'https://raw.githubusercontent.com/Duvey314/austin-green-energy-predictor/master/Resources/Output/Webberville_Solar_2017-2020_MWH.csv'
df = pd.read_csv(url)
df = df.drop(columns=('Weather_Description'))
df = df.drop(columns=('Year'))
df = df.drop(columns=('Month'))
df = df.drop(columns=('Day'))
df = df.drop(columns=('Hour'))
df = df.drop(columns=('Date_Time'))

# create date+hour index 
date_list = pd.date_range(start='01/01/2017', end='31/07/2020')
date_list = pd.to_datetime(date_list)
hour_list = []
for nDate in date_list:
    for nHour in range(24):
        tmp_timestamp = nDate+timedelta(hours=nHour)
        hour_list.append(tmp_timestamp)
date_list = pd.to_datetime(hour_list) 

df['hour_list'] = date_list[:-1]
df = df.set_index('hour_list')

# train, val, test datasets
df_train = df[0:365*24]
df_val = df[365*24:365*24*2]
df_test = df[365*24*2:365*24*3]
df_test_y = df_test[168:]
# split into input-output pairs
train_x, train_y = create_datasets(df_train)
val_x, val_y = create_datasets(df_val)
val_x, scaler_val_x = normalize_data(val_x)
val_y, scaler_val_y =normalize_data(val_y)
test_x, test_y = create_datasets(df_test)
test_x, scaler_test_x = normalize_data(test_x)
test_y, scaler_test_y = normalize_data(test_y)

# training data labels scaler 
scaler_train_y = preprocessing.MinMaxScaler().fit(train_y)

# create ensemble subsets - subsamping
df_ensemble = create_ensemble_datasets(df_train,B=3)

# EnCQR LSTM

In [ ]:
def fit_EnCQR_models(train, testX, testY, trainer_list, B, index, alpha):
  """ fit B models on B subsets created from train data
  Parameters
  ----------
  trainX - training obs.
  trainY - training labels
  trainer_list - network/training parameters
  B      - no. ensemble models 

  Returns
  -------
  f_hat_b_agg     - aggregated leave-one-out predictions for Sb
  ensemble_models - trained models
  indx            - array of Sb indices
  """
  f_hat_b_agg_low = np.zeros((len(train[index[0]][0]),24,B))
  f_hat_b_agg_mean = np.zeros((len(train[index[0]][0]),24,B))
  f_hat_b_agg_high = np.zeros((len(train[index[0]][0]),24,B))
  ensemble_models = []

  # dict containing LOO predictions
  dct_lo = {}
  dct_mean = {}
  dct_hi = {}
  for key in index:
    dct_lo['pred_%s' % key] = []
    dct_mean['pred_%s' % key] = []
    dct_hi['pred_%s' % key] = []

  # training a model for each sub set Sb
  for b in range(B):
    f_hat_b = LSTM_stateful(trainer_list['parameters'])
    f_hat_b = train_model(f_hat_b,train[index[b]][0],train[index[b]][1], testX, testY, trainer_list,early_stop=True)
    ensemble_models.append(f_hat_b)
    # Leave-one-out predictions for each Sb
    indx_LOO = index[np.arange(len(index))!=b]
    for i in range(len(indx_LOO)):
        pred = predict_stateful(train[indx_LOO[i]][0],train[indx_LOO[i]][1],f_hat_b,f_hat_b.batch_s,pad=True)
        pred_inv = np.zeros((pred.shape[0],pred.shape[1],pred.shape[-1]))
        for j in range(pred_inv.shape[-1]):
          pred_inv[:,:,j] = trainer_list['scaler'].inverse_transform(pred[:,:,j])
        dct_lo['pred_%s' %indx_LOO[i]].append(pred_inv[:,:,0])
        dct_mean['pred_%s' %indx_LOO[i]].append(pred_inv[:,:,1])
        dct_hi['pred_%s' %indx_LOO[i]].append(pred_inv[:,:,-1])
  for b in range(B):
    f_hat_b_agg_low[:,:,b] = np.mean(dct_lo['pred_%s' %b],axis=0) 
    f_hat_b_agg_mean[:,:,b] = np.mean(dct_mean['pred_%s' %b],axis=0) 
    f_hat_b_agg_high[:,:,b] = np.mean(dct_hi['pred_%s' %b],axis=0)  

  # residuals
  epsilon = []
  epsilon_hi=[]
  for j in range(B):
    trainY_inv = train[j][1] 
    for i in range(trainY_inv.shape[0]):
      e_low,e_high = asym_nonconformity(label=trainY_inv[i,:],low=f_hat_b_agg_low[i,:,j],high=f_hat_b_agg_high[i,:,j])
      epsilon.append(e_low)
      epsilon_hi.append(e_high)
  epsilon = np.ndarray.flatten(np.array(epsilon))
  epsilon_hi = np.ndarray.flatten(np.array(epsilon_hi))

  # predict test data
  f_hat_t_low = np.zeros((testX.shape[0],testY.shape[1],B))
  f_hat_t_mean = np.zeros((testX.shape[0],testY.shape[1],B))
  f_hat_t_high = np.zeros((testX.shape[0],testY.shape[1],B))
  for b,model in enumerate(ensemble_models):
   test_pred = predict_stateful(testX,testY,model,model.batch_s,pad=True)
  f_hat_t_low[:,:,b] = trainer_list['scaler'].inverse_transform(test_pred[:,:,0])
  f_hat_t_mean[:,:,b] = trainer_list['scaler'].inverse_transform(test_pred[:,:,1])
  f_hat_t_high[:,:,b] = trainer_list['scaler'].inverse_transform(test_pred[:,:,-1])

  # construct PI
  prediction_interval = np.zeros((testY.shape[0],testY.shape[1],3))
  f_hat_t_agg_low = np.zeros((testY.shape[0],testY.shape[1]))
  f_hat_t_agg_mean = np.zeros((testY.shape[0],testY.shape[1]))
  f_hat_t_agg_hi = np.zeros((testY.shape[0],testY.shape[1]))
  org_prediction_interval = np.zeros((testY.shape[0],testY.shape[1],3))

  # aggregate all B predictions for test data
  for day in range(testY.shape[0]):   
    f_hat_t_agg_low[day,:] = np.mean(f_hat_t_low[day,:,:],axis=-1)
    f_hat_t_agg_mean[day,:] = np.mean(f_hat_t_mean[day,:,:],axis=-1)
    f_hat_t_agg_hi[day,:] = np.mean(f_hat_t_high[day,:,:],axis=-1)

    for hour in range(testY.shape[1]):
      # asymetric conformity score index
      index = int(np.ceil((1 - alpha / 2) * (len(epsilon) + 1))) - 1
      index = min(max(index, 0), len(epsilon) - 1)

      # aggregate predictions
      f_quantile_low = np.mean(f_hat_t_low[day,hour,:])
      f_quantile_mean = np.mean(f_hat_t_mean[day,hour,:])
      f_quantile_high = np.mean(f_hat_t_high[day,hour,:])

      # original PI, before conformalization 
      org_prediction_interval[day,hour,0],org_prediction_interval[day,hour,1],org_prediction_interval[day,hour,-1] = f_quantile_low,f_quantile_mean,f_quantile_high

      # conformalization
      e_quantile_lo = np.sort(epsilon)
      e_quantile_hi = np.sort(epsilon_hi)
      e_quantile_lo = e_quantile_lo[index]
      e_quantile_hi = e_quantile_hi[index]  
      prediction_interval[day,hour,0] = f_quantile_low - e_quantile_lo
      prediction_interval[day,hour,1] = f_quantile_mean
      prediction_interval[day,hour,-1] = f_quantile_high + e_quantile_hi

    # update epsilon (s = 24) - TODO: include s as an input variable
    e_lo,e_hi = asym_nonconformity(label=testY[day,:],low=f_hat_t_agg_low[day,:],high=f_hat_t_agg_hi[day,:])
    epsilon = np.delete(epsilon,slice(0,24,1))
    epsilon_hi = np.delete(epsilon_hi,slice(0,24,1))
    epsilon = np.append(epsilon,e_lo)
    epsilon_hi = np.append(epsilon_hi,e_hi)
  return prediction_interval


### The model parameters  - LSTM
n_feat = 6
cells = 89
n_lags = 168
quantiles = [0.09,0.5,0.89]
n_layers = 1
l2_lambda = 0.005
statef = True
batch_size = 7
parameters = {'cells':cells,'n_feat':n_feat,'n_lags':n_lags,'quantiles':quantiles,'n_layers':n_layers,'l2':l2_lambda,'statef':statef,'batch_s':batch_size}

### The taining parameters
n_epochs=100
lr = 0.0009 
optimizer = keras.optimizers.Adam(learning_rate = lr)
loss_function = PinballLoss(quantiles)
trainer_params_list = {'parameters':parameters,'batch_size': batch_size,'epoch_num':n_epochs, 'optimizer': optimizer,'loss_function':loss_function,'lr':lr,'patience':10,'delta':0.01,'scaler':scaler_train_y}

# PREDICT
pred = fit_EnCQR_models(df_ensemble, test_x, test_y, trainer_params_list, 3, np.array([0,1,2]), 0.10)
df_pred = pd.DataFrame(data=pred.reshape(pred.shape[0]*pred.shape[1],pred.shape[-1]),columns=['0.05','0.50','0.95'],index=df_test[168:].index)
c,l = compute_coverage_len(test_y,pred[:,:,0],pred[:,:,-1],verbose=True)
plt.rcParams["figure.figsize"] = (20,5)
df_pred[1000:1168].plot()
df_test_y['MWH'][1000:1168].plot()